In [1]:
import stockutils
import pandas_datareader.data as web, re, sqlite3, datetime

In [120]:
class StockData:
    def __init__(self, se = None, update = False):
        self.company_table = stockutils.company_table(se = se, update = update)
        self.se = se
   
    def lookup_ticker(self, company_name, sql = True, lookup = False):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        if isinstance(company_name,str):
            company_name = [company_name]
        result_table, conn = [], None 
        if not sql:
            for name in company_name:
                for name in [name, name.upper()]:
                    query = r".*" + r".*".join(list(name)) + r".*"
                    for row in self.company_table.keys():
                        match = re.search(query,row)
                        if match:
                            ticker = self.lookup_ticker(match.group(), lookup = True)
                            if ticker:
                                result_table.extend(ticker)
        else:
            conn = sqlite3.connect('data/completecompanylist.sql')
            c = conn.cursor()
            for name in company_name:
                if self.se: 
                    query = (name, self.se,)
                    rows = c.execute('SELECT symbol,name,se FROM companytable WHERE name LIKE ? AND se = ?;', query).fetchall()
                else:
                    query = (name,)
                    rows = c.execute('SELECT symbol,name,se FROM companytable WHERE name LIKE ?;', query).fetchall()
                if rows:
                    result_table.extend(rows)
                else:
                    if not lookup:
                        print('Switching to table for ' + name + ' lookup.')
                        found = self.lookup_ticker(name, sql = False)
                        if found:
                            for item in found:
                                result_table.append(item)
                        else:
                            result_table.append(('N/A', name))
            conn.close()
        return result_table
        
    def ipo_year(self,company_name, sql = True):
        if isinstance(company_name,str):
            company_name = [company_name]
        list_years = []
        conn = sqlite3.connect('data/completecompanylist.sql')
        c = conn.cursor()
        for name in company_name:
            query = (name, )
            list_years.extend(c.execute('SELECT ipoyear, symbol FROM companytable WHERE name LIKE ?;', query).fetchall())
        return list_years
            
#       def stock_price(self, ticker, date):
            
        
        

In [25]:
#update
w = StockData(update = True)

['lastsale', 'ipoyear', 'sector', 'summaryquote', 'industry', 'marketcap', 'symbol', 'name']
['lastsale', 'ipoyear', 'sector', 'summaryquote', 'industry', 'marketcap', 'symbol', 'name']
['lastsale', 'ipoyear', 'sector', 'summaryquote', 'industry', 'marketcap', 'symbol', 'name']


In [96]:
w = StockData()

In [113]:
x = StockData('nasdaq')

In [98]:
y = StockData('amex')

In [121]:
z = StockData('nyse')

In [100]:
x.lookup_ticker('NVIDIA Corporation')

[('NVDA', 'NVIDIA Corporation', 'nasdaq')]

In [107]:
x.lookup_ticker(['Intel Corporation', 'Nvidia', 'Apple', 'Microsoft', 'Becton Dickinson'])

Switching to table for Nvidia lookup.
Switching to table for Apple lookup.
Switching to table for Microsoft lookup.
Switching to table for Becton Dickinson lookup.


[('INTC', 'Intel Corporation', 'nasdaq'),
 ('NVDA', 'NVIDIA Corporation', 'nasdaq'),
 ('AMCC', 'Applied Micro Circuits Corporation', 'nasdaq'),
 ('AGTC', 'Applied Genetic Technologies Corporation', 'nasdaq'),
 ('AMAT', 'Applied Materials, Inc.', 'nasdaq'),
 ('AAOI', 'Applied Optoelectronics, Inc.', 'nasdaq'),
 ('AAPL', 'Apple Inc.', 'nasdaq'),
 ('ARCI', 'Appliance Recycling Centers of America, Inc.', 'nasdaq'),
 ('APDN', 'Applied DNA Sciences Inc', 'nasdaq'),
 ('MSFT', 'Microsoft Corporation', 'nasdaq'),
 ('N/A', 'Becton Dickinson')]

In [102]:
z.lookup_ticker("Dick's")

Switching to table for Dick's lookup.


[('DKS', "Dick's Sporting Goods Inc", 'nyse')]

In [103]:
z.lookup_ticker('Becton')

Switching to table for Becton lookup.


[('BSX', 'Boston Scientific Corporation', 'nyse'),
 ('WAB', 'Westinghouse Air Brake Technologies Corporation', 'nyse'),
 ('INF', 'Brookfield Global Listed Infrastructure Income Fund', 'nyse'),
 ('BTZ', 'BlackRock Credit Allocation Income Trust', 'nyse'),
 ('BR', 'Broadridge Financial Solutions, Inc.', 'nyse'),
 ('BHE', 'Benchmark Electronics, Inc.', 'nyse'),
 ('BDX', 'Becton, Dickinson and Company', 'nyse'),
 ('IBM', 'International Business Machines Corporation', 'nyse'),
 ('BGE^B', 'Baltimore Gas & Electric Company', 'nyse'),
 ('BAC', 'Bank of America Corporation', 'nyse'),
 ('BNED', 'Barnes & Noble Education, Inc', 'nyse'),
 ('JBT', 'John Bean Technologies Corporation', 'nyse'),
 ('BIT', 'BlackRock Multi-Sector Income Trust', 'nyse'),
 ('BPI', 'Bridgepoint Education, Inc.', 'nyse')]

In [104]:
w.lookup_ticker(['Intel', 'BD'])

Switching to table for Intel lookup.
Switching to table for BD lookup.


[('OFIX', 'Orthofix International N.V.', 'nasdaq'),
 ('IQNT', 'Inteliquent, Inc.', 'nasdaq'),
 ('CYD', 'China Yuchai International Limited', 'nyse'),
 ('CID',
  'Victory CEMP International High Div Volatility Wtd Index ETF',
  'nasdaq'),
 ('BKN', 'BlackRock Investment Quality Municipal Trust Inc. (The)', 'nyse'),
 ('CSGS', 'CSG Systems International, Inc.', 'nasdaq'),
 ('ICPT', 'Intercept Pharmaceuticals, Inc.', 'nasdaq'),
 ('HSII', 'Heidrick & Struggles International, Inc.', 'nasdaq'),
 ('IP', 'International Paper Company', 'nyse'),
 ('ATNI', 'ATN International, Inc.', 'nasdaq'),
 ('MATW', 'Matthews International Corporation', 'nasdaq'),
 ('AXE', 'Anixter International Inc.', 'nyse'),
 ('CXRX', 'Concordia International Corp.', 'nasdaq'),
 ('ORI', 'Old Republic International Corporation', 'nyse'),
 ('ITGR', 'Integer Holdings Corporation', 'nyse'),
 ('GNUS', 'Genius Brands International, Inc.', 'nasdaq'),
 ('VIDI', 'Vident International Equity Fund', 'nasdaq'),
 ('AEL', 'American Equity

In [115]:
x.ipo_year('Intel Corporation')

[('n/a', 'Intel Corporation')]

In [122]:
z.ipo_year(['Blackrock Core Bond Trust', 'BlackRock Credit Allocation Income Trust'])

[('2001', 'BHK'), ('2006', 'BTZ')]